In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("spotify_millsongdata.csv")

In [3]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [5]:
df.shape

(57650, 4)

In [6]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [7]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [8]:
df.head(10)

,artist,song,text
0,LL Cool J,Apple Cobbler,Uh uh uh uh-huh \r\nUh uh that joint is hot b...
1,Joni Mitchell,Harlem In Havana,At the far end of the midway \r\nBy the doubl...
2,Westlife,No More Heroes,When the waves are crashing down \r\nPulling ...
3,Hank Williams Jr.,"Lawyers, Guns, And Money",I went home with the waitress \r\nThe way I a...
4,Peter Cetera,Queen Of The Masquerade Ball,She lays her cards out on the table \r\nShe a...
5,Kinks,Holloway Jail,"They took my baby, to Holloway jail, \r\nThey..."
6,Ellie Goulding,Keep On Dancin',[Verse 1] \r\nPeople like to talk because the...
7,Young Jeezy,All White Everything,Married to the game \r\nShe even got a weddin...
8,Tori Amos,Nautical Twilight,As the day gave way \r\nTo Nautical Twilight ...
9,Devo,All Of Us,I left home with a friend of mine \r\nGone tw...


In [9]:
df['text'][0]

"Uh uh uh uh-huh  \r\nUh uh that joint is hot baby!  \r\n  \r\nLights, camera, action, hold up  \r\nYou know my style, I been blowed up  \r\nPaper was young, now it's growed up  \r\nStacks so thick it's hard to fold up  \r\nYo b, find another rubber band in the truck  \r\nCount up the money, I'ma stand in the cut  \r\nStroll in the party and I toast Cris' up  \r\nTell that a muh-hucca gets this up  \r\nShake that cookie like what like what  \r\nToss me a drop it's like lightning struck  \r\nLook at that apple cobbler butt  \r\nWhatchu want to do, whatchu think? Want cut  \r\nNBA Live in my truck  \r\nParkin' lot like all jammed up  \r\nIf there's beef it's best you duck  \r\nI'm gon' eat 'til I'm filled up  \r\n  \r\n[Chorus]  \r\nThrow it to me, that apple cobbler  \r\nBaby it, can I see (can I see) that apple pie?  \r\nI said throw it to me, that apple cobbler  \r\nBaby it, can I get (hey) can I get it deep fried?  \r\n  \r\nSo much sugar it's makin' my head rush  \r\nTell me what th

In [10]:
# df = df.sample(5000)

In [11]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [12]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [13]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [14]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [17]:
similarity[0]

array([1.        , 0.0170025 , 0.00553629, ..., 0.02544314, 0.04908935,
       0.00958524])

In [24]:
df[df['song'] == 'Apple Cobbler']

,artist,song,text
0,LL Cool J,Apple Cobbler,uh uh uh uh-huh uh uh that joint is hot babi !...


In [25]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [26]:
recommendation('Apple Cobbler')

['Go Out',
 'Turning Circles',
 'Every Sip',
 'Just Wanna Be With You',
 'One More Bite Of The Apple',
 'Islands In The Stream',
 'Rock Show',
 'Sway',
 'You Are The Sunshine Of My Life',
 'The Apple Tree',
 'Sweetheart',
 'Marc Live',
 "Makin' Love",
 'Just The Two Of Us',
 'Tilt Ya Head Back',
 'Complicated',
 'Hot Revolver',
 'Can I Have It Like That',
 'I Am The Bullgod',
 'Still Standing']

In [27]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))